# Jupyter Dashによるダッシュボードのデモ（折れ線グラフに四分位値を追加）

In [4]:
# japanize_matplotlibが未インストール状態の場合に、コメント解除して本セルを単独で実行する（先頭に!が必要）
#!pip install japanize-matplotlib

In [5]:
# JupyterDash他が未インストール状態の場合に、コメント解除して本セルを単独で実行する（先頭に!が必要）
#!pip install jupyter_dash
#!pip install --upgrade plotly
#!pip install dash_bootstrap_components

In [6]:
# Googleドライブがマウントされていない場合に、2行のコメント解除して本セルを単独で実行する
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
# matplotlibで日本語を使えるようにする
import japanize_matplotlib

In [8]:
import pandas as pd
import numpy as np

In [9]:
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
from dash import html
from dash import dcc
from dash import dash_table
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from dash_table import FormatTemplate
from dash.dash_table.Format import Format

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  # This is added back by InteractiveShellApp.init_path()


## データファイルを読み込む

In [10]:
 df1 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/都道府県各種指標.xlsx',sheet_name='data')
 df1 = df1.assign(ch_year = df1['年度'].apply(lambda x: str(x)+'年'))
 df1 = df1.assign(id_pref = df1['地域コード']+df1['都道府県'])
 df1.sort_values('年度',inplace=True)
 df1.head(3)

,年度,地域コード,都道府県,総人口,総人口（男）,総人口（女）,日本人人口,日本人人口（男）,日本人人口（女）,15歳未満人口,...,光熱・水道費（二人以上の世帯）,家具・家事用品費（二人以上の世帯）,被服及び履物費（二人以上の世帯）,保健医療費（二人以上の世帯）,交通・通信費（二人以上の世帯）,教育費（二人以上の世帯）,教養娯楽費（二人以上の世帯）,その他の消費支出（二人以上の世帯）,ch_year,id_pref
563,2007,R47000,沖縄県,1374000,674000,700000,1367000,670000,697000,248000,...,20148,6470,6682,9622,25297,10303,17777,51202,2007年,R47000沖縄県
467,2007,R39000,高知県,783000,367000,415000,780000,366000,414000,97000,...,20144,9908,9979,10702,35384,14129,25772,72034,2007年,R39000高知県
479,2007,R40000,福岡県,5061000,2395000,2666000,5021000,2375000,2646000,701000,...,20774,9772,16896,15624,40340,13307,34726,68618,2007年,R40000福岡県


In [11]:
# グループ化変数以外は数値変数に絞り込む
df301 = df1.drop(columns=['地域コード','都道府県'])
# 欠損値はゼロに置き換える
df302 = df301.fillna(value=0)
df302.head(3)

,年度,総人口,総人口（男）,総人口（女）,日本人人口,日本人人口（男）,日本人人口（女）,15歳未満人口,15歳未満人口（男）,15歳未満人口（女）,...,光熱・水道費（二人以上の世帯）,家具・家事用品費（二人以上の世帯）,被服及び履物費（二人以上の世帯）,保健医療費（二人以上の世帯）,交通・通信費（二人以上の世帯）,教育費（二人以上の世帯）,教養娯楽費（二人以上の世帯）,その他の消費支出（二人以上の世帯）,ch_year,id_pref
563,2007,1374000,674000,700000,1367000,670000,697000,248000,127000,121000,...,20148,6470,6682,9622,25297,10303,17777,51202,2007年,R47000沖縄県
467,2007,783000,367000,415000,780000,366000,414000,97000,50000,48000,...,20144,9908,9979,10702,35384,14129,25772,72034,2007年,R39000高知県
479,2007,5061000,2395000,2666000,5021000,2375000,2646000,701000,359000,342000,...,20774,9772,16896,15624,40340,13307,34726,68618,2007年,R40000福岡県


In [12]:
# 要約統計量による集計
df310 = df302.groupby(['ch_year']).describe().stack(level=1)
df310.head(3)

年度           総人口        総人口（男）        総人口（女）         日本人人口  \
ch_year                                                                         
2007年   count    47.0  4.700000e+01  4.700000e+01  4.700000e+01  4.700000e+01   
        mean   2007.0  2.724170e+06  1.328191e+06  1.395851e+06  2.688191e+06   
        std       0.0  2.635795e+06  1.312330e+06  1.324592e+06  2.584537e+06   

                   日本人人口（男）      日本人人口（女）        15歳未満人口     15歳未満人口（男）  \
ch_year                                                                   
2007年   count  4.700000e+01  4.700000e+01      47.000000      47.000000   
        mean   1.311383e+06  1.376915e+06  367957.446809  188574.468085   
        std    1.287325e+06  1.298075e+06  341271.405282  174942.194761   

                  15歳未満人口（女）  ...  食料費（二人以上の世帯）  住居費（二人以上の世帯）  \
ch_year                       ...                               
2007年   count      47.000000  ...     47.000000     47.000000   
        mean   179340.425532  ...  67765.340426  18782.808511   
        std    166285.934540  ...   5381.216758   4385.865115   

               光熱・水道費（二人以上の世帯）  家具・家事用品費（二人以上の世帯）  被服及び履物費（二人以上の世帯）  \
ch_year                                                               
2007年   count        47.000000          47.000000         47.000000   
        mean      21943.638298        9706.468085      13596.914894   
        std        2225.036213        1325.678987       2260.165806   

               保健医療費（二人以上の世帯）  交通・通信費（二人以上の世帯）  教育費（二人以上の世帯）  教養娯楽費（二人以上の世帯）  \
ch_year                                                                        
2007年   count       47.000000        47.000000     47.000000       47.000000   
        mean     12768.021277     38077.595745  12858.446809    31742.659574   
        std       1625.384924      6722.290777   3687.609682     5317.637532   

               その他の消費支出（二人以上の世帯）  
ch_year                           
2007年   count          47.000000  
        mean        74304.680851  
        std         12501.731672  

[3 rows x 106 columns]

In [13]:
df311 = df310.reset_index()
df312 = df311[df311['level_1'].str.contains("25%|50%|75%")]
df313 = df312.rename(columns={'level_1':'都道府県'})
df313.head(3)

,ch_year,都道府県,年度,総人口,総人口（男）,総人口（女）,日本人人口,日本人人口（男）,日本人人口（女）,15歳未満人口,...,食料費（二人以上の世帯）,住居費（二人以上の世帯）,光熱・水道費（二人以上の世帯）,家具・家事用品費（二人以上の世帯）,被服及び履物費（二人以上の世帯）,保健医療費（二人以上の世帯）,交通・通信費（二人以上の世帯）,教育費（二人以上の世帯）,教養娯楽費（二人以上の世帯）,その他の消費支出（二人以上の世帯）
4,2007年,25%,2007.0,1159500.0,552000.0,606500.0,1152500.0,549500.0,603500.0,159000.0,...,65230.0,16095.5,20435.5,8913.0,12304.0,11728.5,32988.5,10533.0,28628.5,67147.0
5,2007年,50%,2007.0,1733000.0,809000.0,924000.0,1728000.0,808000.0,920000.0,248000.0,...,67306.0,19049.0,21481.0,9796.0,13445.0,12615.0,38368.0,12127.0,30907.0,72034.0
6,2007年,75%,2007.0,2758500.0,1329500.0,1429000.0,2720000.0,1310500.0,1409000.0,373000.0,...,70044.0,21598.0,22949.0,10357.0,14820.5,14022.5,42192.0,14133.5,34257.5,79054.0


表示候補の列名のリストを作成する

In [14]:
my_list = df1.columns
to_remove = ['年度','地域コード','都道府県','ch_year','id_pref']
remained_list = [i for i in my_list if i not in to_remove]

Dash - Plotlyでの描画の準備

In [15]:
# Dash表示用の設定：グラフ１（分布グラフ）
default_kpi_1 = '総人口'
plot_type_1 = ['箱ひげ図','散布図','バイオリン']
default_plot_1 = '箱ひげ図'
# Dash表示用の設定：グラフ２（基本グラフ）
default_kpi_2 = '年平均気温'
plot_type_2 = ['棒グラフ','折れ線グラフ']
default_plot_2 = '折れ線グラフ'
default_pref_2 = '神奈川県'
# Dash表示用の設定：グラフ３（データテーブル）
default_kpi_3 = '総人口'

appという「箱」を作る

In [16]:
# JupyterDashの「箱」を作る
app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP],
                  meta_tags=[{'name': 'viewport',
                              'content': 'width=device-width, initial-scale=1.0'}]) # レスポンシブ対応のためのメタタグ

グラフ１（分布グラフ）のコンテンツの定義

In [17]:
# ドロップダウンリストを定義する
content1 = dcc.Dropdown(
    id = 'dropdown-for-kpi-1', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in remained_list], # 値を選択する列を指定
    value = default_kpi_1 # デフォルト値を設定
)
content2 = dcc.Dropdown(
    id = 'dropdown-for-plot-1', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in plot_type_1], # 値を選択する列を指定
    value = default_plot_1 # デフォルト値を設定
)
content3 = dcc.Graph(id='box-plot')

グラフ２（基本グラフ）のコンテンツの定義

In [18]:
content4 = dcc.Dropdown(
    id = 'dropdown-for-pref-2', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in df1['都道府県'].unique()], # 都道府県を指定
    value = default_pref_2 # デフォルト値を設定
)
content5 = dcc.Dropdown(
    id = 'dropdown-for-kpi-2', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in remained_list], # 指標値の種類を指定
    value = default_kpi_2 # デフォルト値を設定
)
content6 = dcc.Dropdown(
    id = 'dropdown-for-plot-2', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in plot_type_2], # グラフのタイプを指定
    value = default_plot_2 # デフォルト値を設定
)
content7 = dcc.Graph(id='basic-plot')

グラフ３（データテーブル）のコンテンツの定義

In [19]:
# 表示する列を定義する
columns1 = [
    dict(id='都道府県',name='都道府県'),
    dict(id='2007年', name='2007年度'),
    dict(id='2008年', name='2008年度'),
    dict(id='2009年', name='2009年度'),
    dict(id='2010年', name='2010年度'),
    dict(id='2011年', name='2011年度'),
    dict(id='2012年', name='2012年度'),
    dict(id='2013年', name='2013年度'),
    dict(id='2014年', name='2014年度'),
    dict(id='2015年', name='2015年度'),
    dict(id='2016年', name='2016年度'),
    dict(id='2017年', name='2017年度'),
    dict(id='2018年', name='2018年度')
]
# ドロップダウンリストを定義する
content8 = dcc.Dropdown(
    id = 'dropdown-for-kpi-3', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in remained_list], # 値を選択する列を指定
    value = default_kpi_3 # デフォルト値を設定
)
# データテーブルを定義する
content9 = html.Div(id='datatable-paging',children=[]) # id名はコールバックで必要

画面のレイアウトを設定する

In [20]:
# 画面のレイアウトを定義する
app.layout = dbc.Container([
    dbc.Row([
             dbc.Col(html.H1("日本の都道府県の各種指標",
                             className='text-center text-primary, mb-4'), # 中寄せでタイトルの下にスペース
                     width=12) # 全幅(12列分)の幅をとる
    ]),
    dbc.Row([
             dbc.Col([
                      html.P("各年度の指標値の分布",style={'textDecoration': 'underline'}),
                      html.Div(content1,style={'width':'30%','display':'inline-block','margin-right':10}),
                      html.Div(content2,style={'width':'30%','display':'inline-block','margin-right':10}),
                      content3
             ],width={'size':10, 'order':1})
             ], justify='around', align='center'), 
    dbc.Row([
            dbc.Col([
                     html.P("都道府県の指標値の推移",style={'textDecoration': 'underline'}),
                     html.Div(content4,style={'width':'30%','display':'inline-block','margin-right':10}),
                     html.Div(content5,style={'width':'30%','display':'inline-block','margin-right':10}),
                     html.Div(content6,style={'width':'30%','display':'inline-block','margin-right':10}),
                     content7
            ],width={'size':10, 'order':1}), # 左に余白を設けたい場合は'offset':列数を指定する。orderは図の順序
#             xs=12, sm=12, md=12, lg=5, xl=5), # 小さいサイズの画面の場合は幅を12にして縦に並ぶようにする)
    ], justify='around', align='center'), 
    dbc.Row([
             dbc.Col([
                    html.P("指標の数値一覧",style={'textDecoration': 'underline'}),
                    html.Div(content8, style={'width': '30%', 'display': 'inline-block','margin-right': 10}),
                    content9      
             ], width={'size':'auto', 'order':1}) # width="auto"はコンテンツの自然な幅
    ] ,justify='around', align='center')
], fluid=True) # fluid=Trueで端の空白をなくす

グラフ１（分布グラフ）のコールバックと実行関数

In [21]:
# コールバックを定義する（ドロップダウンリストで選択された値をもとに図を描画する）
# どこから入力し、どこへ出力するかを指定する
@app.callback(
    Output(component_id='box-plot',component_property='figure'),
    [Input(component_id='dropdown-for-kpi-1',component_property='value'),
     Input(component_id='dropdown-for-plot-1',component_property='value')]
)
def generate_chart(inval1,inval2): # 行間をあけずに関数部分を書く（関数名は何でも良い）
  if inval2 == '箱ひげ図':
    fig = go.Figure()
    fig.add_trace(go.Box(x=df1['ch_year'],y=df1[inval1],text=df1['都道府県']))
    fig.update_layout(boxgroupgap=0.1,height = 450,margin=dict(l=20,r=20,t=20,b=20),
                      paper_bgcolor="GhostWhite",font={"family":"Meiryo","size":16})
  elif inval2 == '散布図':
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df1['ch_year'],y=df1[inval1],
                      mode='markers',text=df1['都道府県']))
    fig.update_traces(marker=dict(size=12,line=dict(width=2,color='DarkSlateGray')),
                      selector=dict(mode='markers'))
    fig.update_layout(boxgroupgap=0.1,height = 450,margin=dict(l=20,r=20,t=20,b=20),
                      paper_bgcolor="GhostWhite",font={"family":"Meiryo","size":16})
  else:
    fig = go.Figure()
    fig.add_trace(go.Violin(x=df1['ch_year'],y=df1[inval1],text=df1['都道府県']))
    fig.update_layout(boxgroupgap=0.1,height = 450,margin=dict(l=20,r=20,t=20,b=20),
                      paper_bgcolor="GhostWhite",font={"family":"Meiryo","size":16})
  return fig

グラフ２（基本グラフ）のコールバックと実行関数

In [22]:
# コールバックを定義する（ドロップダウンリストで選択された値をもとに図を描画する）
# どこから入力し、どこへ出力するかを指定する
@app.callback(
    Output(component_id='basic-plot',component_property='figure'),
    [Input(component_id='dropdown-for-pref-2',component_property='value'),
     Input(component_id='dropdown-for-kpi-2',component_property='value'),
     Input(component_id='dropdown-for-plot-2',component_property='value')]
)
def generate_chart(inval1,inval2,inval3): # 行間をあけずに関数部分を書く（関数名は何でも良い）
  df201 = df1[df1['都道府県']==inval1] # ドロップダウンリストで指定された都道府県に絞り込む
  if inval3 == '折れ線グラフ':
    df202 = pd.concat([df201,df313])
    fig = px.line(df202, x='ch_year', y=inval2,color='都道府県',markers=True) # 折れ線グラフを描画する
    fig.update_layout(height = 450,margin=dict(l=20,r=20,t=20,b=20),
                      paper_bgcolor="GhostWhite",font={"family":"Meiryo","size":20})
  else:
    fig = px.bar(df201, x='年度', y=inval2) # 棒グラフを描画する
    fig.update_layout(height = 450,margin=dict(l=20,r=20,t=20,b=20),
                      paper_bgcolor="GhostWhite",font={"family":"Meiryo","size":20})
  return fig

グラフ３（データテーブル）のコールバックと実行関数

In [23]:
# データテーブルを描画する関数を定義する
# 一行おきに色のコントラストをつけたデザインにする
def create_dash_table(df):
  return dash_table.DataTable(
      data=df.to_dict('records'),
      columns=columns1,
      style_cell={'fontsize':20, 'font-family':'IPAexGothic'},
      style_cell_conditional=[
                              {
                                  'if': {'column_id':c},
                                  'textAlign': 'left'
                              } for c in ['都道府県']
      ],
      style_data={
      'color':'black',
      'backgroundColor':'white'
      },
      style_data_conditional=[
                          {
                              'if': {'row_index':'odd'},
                              'backgroundColor': 'rgb(220,220,220)',
                          }
      ],
      style_header={
          'backgroundColor':'rgb(210,210,210)',
          'color':'black',
          'fontWeight':'bold'
  })

In [24]:
# コールバックを定義する（ドロップダウンリストで選択された値をもとに表を作成する）
# どこから入力し、どこへ出力するかを指定する
@app.callback(
    Output(component_id='datatable-paging',component_property='children'),
    [Input(component_id='dropdown-for-kpi-3',component_property='value')]
)
def update_table(inval1): # 行間をあけずに関数部分を書く（関数名は何でも良い）
    df502 = df1.pivot(index='id_pref', columns='ch_year', values=inval1)
    df502.reset_index(inplace=True)
    df502 = df502.assign(都道府県=df502['id_pref'].apply(lambda x: x[6:]))
    return create_dash_table(df502)

In [25]:
# サーバーを起動する
if __name__ == '__main__':
    app.run_server(debug=True, port=3000)

Dash app running on:


<IPython.core.display.Javascript object>